# Specification

## Project 1 -Predict secondary protein structure given the sequence. 

### Completion requirements:

- Reimplement the network described by Qian and Sejnowski in 1988


- Test and compare your accuracy - using their data


- Implement a single improvement, such as profiling


- Test and compare your accuracy again


- Does the model get similar accuracy on unseen datasets?


- Extend your work to other methods, e.g. can large language models help? How about SVMs?

# Import Modules

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf


# Import Dataset

In [4]:
# Define a function to convert amino acids and structures to one-hot

def one_hot_encode(seq, vocab):
    """One-hot encode a sequence based on a given vocabulary."""
    one_hot = np.zeros((len(seq), len(vocab)), dtype=np.float32)
    for i, char in enumerate(seq):
        if char in vocab:
            one_hot[i, vocab.index(char)] = 1.0
    return one_hot

def prepare_data(filepath, window_size=13):
    sequences = []
    structures = []
    current_seq = []
    current_struct = []
    processing_sequence = False  # Track when inside a sequence block

    with open(filepath, 'r') as file:
        for line in file:
            line = line.strip()
            if line == '<>':  # Toggle processing flag
                if processing_sequence:  # We are ending a sequence block
                    if current_seq and current_struct:
                        seq_encoded = one_hot_encode(current_seq, aa_vocab)
                        struct_encoded = one_hot_encode(current_struct, structure_vocab)

                        # Apply sliding window
                        for i in range(len(seq_encoded) - window_size + 1):
                            window = seq_encoded[i:i + window_size]
                            label = struct_encoded[i + window_size // 2]
                            sequences.append(window)
                            structures.append(label)

                    current_seq = []
                    current_struct = []
                processing_sequence = not processing_sequence
                continue

            elif 'end' in line:  # Generalized handling for any 'end' marker
                continue  # Just skip this line, do not end processing sequence

            if processing_sequence:
                parts = line.split()
                if len(parts) != 2:
                    continue  # Skip malformed lines or lines that do not fit expected format
                current_seq.append(parts[0])
                current_struct.append(parts[1])

    return np.array(sequences), np.array(structures)


# Define your vocabularies
aa_vocab = 'ACDEFGHIKLMNPQRSTVWY_'  # 20 amino acids + 1 for gap/unknown
structure_vocab = 'he_'  # h for helix, e for sheet, _ for coil

# Example paths, replace with your actual file pathsin
train_path = 'Q_and_s_data/protein-secondary-structure.train.txt'
test_path = 'Q_and_s_data/protein-secondary-structure.test.txt'

x_train, y_train = prepare_data(train_path)
x_test, y_test = prepare_data(test_path)

In [5]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(8110, 13, 21) (8110, 3)
(1714, 13, 21) (1714, 3)


In [6]:
def test_data_loading(path):
    sample_path = path
    sequences, structures = prepare_data(sample_path)
    print("Number of sequences loaded:", len(sequences))
    print("Number of structure labels loaded:", len(structures))
    assert len(sequences) == len(structures), "Each sequence should have a corresponding structure label."
    print("Data loading test passed.")

test_data_loading(train_path)
test_data_loading(test_path)


Number of sequences loaded: 8110
Number of structure labels loaded: 8110
Data loading test passed.
Number of sequences loaded: 1714
Number of structure labels loaded: 1714
Data loading test passed.


In [7]:
def test_sequence_format(path):
    sample_path = path
    sequences, structures = prepare_data(sample_path)
    print("Shape of one sequence:", sequences[0].shape)
    print("Shape of one structure label:", structures[0].shape)
    assert sequences[0].shape == (13, 21), "Each sequence window should be (13, 21)."
    assert structures[0].shape == (3,), "Each label should be a one-hot vector of length 3."
    print("Sequence format test passed.")

test_sequence_format(train_path)
test_sequence_format(test_path)

Shape of one sequence: (13, 21)
Shape of one structure label: (3,)
Sequence format test passed.
Shape of one sequence: (13, 21)
Shape of one structure label: (3,)
Sequence format test passed.


In [8]:
def inspect_specific_entries(path):
    sample_path = path
    sequences, structures = prepare_data(sample_path)
    index = 10  # Choose an index to inspect
    print("One-hot encoded sequence at index", index, ":", sequences[index])
    print("One-hot encoded structure label at index", index, ":", structures[index])

inspect_specific_entries(train_path)
inspect_specific_entries(test_path)

One-hot encoded sequence at index 10 : [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
One-hot encoded structure label at index 10 : [0. 0. 1.]
One-hot encoded sequence at index 10 : [[0. 0

In [9]:
def check_end_marker_handling(path):
    sample_path = path
    sequences, structures = prepare_data(sample_path)
    print("Checking last few sequences for proper termination handling...")
    # Display the last few sequences and labels to manually verify
    for i in range(len(sequences)-5, len(sequences)):
        print("Sequence:", sequences[i])
        print("Label:", structures[i])

check_end_marker_handling(train_path)
check_end_marker_handling(test_path)


Checking last few sequences for proper termination handling...
Sequence: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Label: [0. 0. 1.]
Sequence: [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

# Performane Metrics

In [24]:
from sklearn.metrics import confusion_matrix

def calculate_q3_score(y_true, y_pred):
    y_true_classes = np.argmax(y_true, axis=1)
    y_pred_classes = np.argmax(y_pred, axis=1)
    return np.mean(y_true_classes == y_pred_classes)

def calculate_correlation_coefficients(y_true, y_pred):
    """Calculates Pearson correlation coefficients for each class."""
    correlations = []
    for i in range(y_true.shape[1]):  # Assuming y_true is one-hot encoded
        # Extract the actual and predicted values for each class
        true_values = y_true[:, i]
        pred_values = y_pred[:, i]
        
        # Calculate correlation using numpy's corrcoef, which returns a correlation matrix
        if np.std(true_values) != 0 and np.std(pred_values) != 0:
            corr_matrix = np.corrcoef(true_values, pred_values)
            corr_coefficient = corr_matrix[0, 1]  # Extract the off-diagonal value which is the correlation coefficient
            correlations.append(corr_coefficient)
        else:
            correlations.append(0)  # Avoid division by zero in case of constant true or pred values
    return correlations

def matthews_correlation_coefficient(y_true, y_pred):
    """Calculate the Matthews correlation coefficient for each class."""
    MCCs = []
    # Assuming y_true and y_pred are one-hot encoded, decode to single class predictions
    y_true_classes = np.argmax(y_true, axis=1)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    # Get unique classes
    classes = np.unique(y_true_classes)
    
    for class_id in classes:
        # Create binary arrays for each class
        true_binary = (y_true_classes == class_id).astype(int)
        pred_binary = (y_pred_classes == class_id).astype(int)
        
        # Compute the confusion matrix
        tn, fp, fn, tp = confusion_matrix(true_binary, pred_binary).ravel()
        
        # Compute MCC
        mcc_numerator = (tp * tn) - (fp * fn)
        mcc_denominator = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
        mcc = mcc_numerator / mcc_denominator if mcc_denominator != 0 else 0
        MCCs.append(mcc)
    
    return MCCs


# Model Architecture

In [29]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(13, 21)),  # Flatten the input layer to handle a window of 13 amino acids
    tf.keras.layers.Dense(40, activation='relu'),  # Hidden layer with 64 neurons
    tf.keras.layers.Dense(3, activation='linear')  # Output layer for 3 classes, linear activation for MAE loss
])

model.compile(
    optimizer='sgd',  # Using simple Stochastic Gradient Descent
    loss='mean_absolute_error',  # MAE loss for regression-like behavior
    metrics=['mean_absolute_error']  # Track MAE during training
)

# Display model architecture
#model.summary()

c:\Users\Tom\anaconda3\envs\bio-cwk\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


# Train


In [31]:
history = model.fit(
    x_train, y_train,
    epochs=100,  # The number of epochs can be adjusted based on when you observe convergence
    batch_size=32,  # The batch size can be tuned based on your hardware capabilities
    validation_data=(x_test, y_test)  # Use your test set for validation
)

Epoch 1/100
254/254 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2801 - mean_absolute_error: 0.2801 - val_loss: 0.2838 - val_mean_absolute_error: 0.2838
Epoch 2/100
254/254 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2731 - mean_absolute_error: 0.2731 - val_loss: 0.2836 - val_mean_absolute_error: 0.2836
Epoch 3/100
254/254 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2757 - mean_absolute_error: 0.2757 - val_loss: 0.2831 - val_mean_absolute_error: 0.2831
Epoch 4/100
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2753 - mean_absolute_error: 0.2753 - val_loss: 0.2829 - val_mean_absolute_error: 0.2829
Epoch 5/100
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2767 - mean_absolute_error: 0.2767 - val_loss: 0.2834 - val_mean_absolute_error: 0.2834
Epoch 6/100
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2758 - mean_absolute_error: 0.2758 - val_loss: 0.2828 - val_mean_absolute_error: 0.2828
Epoch 7/100
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2763 - mean_absolute_error: 0.2

In [32]:
# Predict on test data
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Calculate Q3 score
q3_score = calculate_q3_score(y_test, y_pred)
print(f"Q3 Score: {q3_score * 100:.2f}%")

mcc_scores = matthews_correlation_coefficient(y_test, y_pred)
for i, mcc in enumerate(mcc_scores):
    print(f"MCC for class {['Alpha', 'Beta', 'Coil'][i]}: {mcc:.3f}")

54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Q3 Score: 57.12%
MCC for class Alpha: 0.117
MCC for class Beta: 0.051
MCC for class Coil: 0.183
